# News Article Fetcher using Kafka, Bytewax, and Upstash Vector DB
---
- Build live system that fetches news artices from live news APIs, parses and formats raw data into our defined formats then serializes and streams messages to Kafka Topic.
- Then, use Bytewas to streamline the messages from Kafka Topic by cleaning, parsing, chunking, embedding, and upserting vectors to a Vector DB, ending with a UI from which we can interact with our DB.

In [37]:
import requests
import json
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
from dotenv import load_dotenv
from pprint import pprint
import logging

In [38]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [39]:

url_everything = ('https://newsapi.org/v2/everything?'
       'q=Data+Science&'
       'from=2024-04-20&'
       'sortBy=popularity&'
       'apiKey=4a107bac530044a4a6689d38744dcc65')

response = requests.get(url_everything)
result = json.loads(response.content)

print(response.json)
print(result)

<bound method Response.json of <Response [200]>>
{'status': 'ok', 'totalResults': 173, 'articles': [{'source': {'id': None, 'name': 'Unstructured.io'}, 'author': None, 'title': 'Understanding What Matters for LLM Ingestion and Preprocessing', 'description': 'Unstructured effortlessly extracts and transforms complex data for use with every major vector database and LLM framework.', 'url': 'https://unstructured.io/blog/understanding-what-matters-for-llm-ingestion-and-preprocessing', 'urlToImage': 'https://framerusercontent.com/images/wA7B5VKN3miwjCmQiCv1tCpc8.jpg', 'publishedAt': '2024-04-21T17:30:54Z', 'content': 'Imagine a world where all of your organization’s unstructured data lies at your fingertips. A mere prompt and your LLM application unlocks deep insights from a labyrinth of documents, emails, and rep… [+20498 chars]'}, {'source': {'id': None, 'name': 'Nautil.us'}, 'author': 'Michael Inzlicht', 'title': 'The Psychology of Getting High–A Lot', 'description': 'Time to extinguish 

In [40]:
url_top_headlines = ('https://newsapi.org/v2/top-headlines?'
    # 'q=Data+Science&'
    'country=us&'
    'from=2024-04-20&'
    # 'sortBy=popularity&'
    'apiKey=4a107bac530044a4a6689d38744dcc65')

response = requests.get(url_top_headlines)
result = json.loads(response.content)

print(response.json)
print(result)

<bound method Response.json of <Response [200]>>
{'status': 'ok', 'totalResults': 34, 'articles': [{'source': {'id': None, 'name': 'New York Post'}, 'author': 'Doree Lewak, Isabel Keane', 'title': "Columbia rabbi warns Jewish students to go home, don't come back to campus because of 'extreme antisemitism' - New York Post ", 'description': 'A prominent rabbi for Columbia University has warned Jewish students to go home and not return to campus because of “extreme antisemitism” at the Ivy League university.', 'url': 'https://nypost.com/2024/04/21/us-news/columbia-rabbi-warns-jewish-students-to-go-home-dont-come-back-to-campus-because-of-extreme-antisemitism/', 'urlToImage': 'https://nypost.com/wp-content/uploads/sites/2/2024/04/comp-1-4.jpg?quality=75&strip=all&w=1024', 'publishedAt': '2024-04-21T17:11:51Z', 'content': 'A prominent rabbi at Columbia University took the drastic step Sunday of warning Jewish students to go home and not return to campus because of extreme antisemitism at th

In [41]:
load_dotenv("../.env")

True

In [42]:
client = NewsApiClient(api_key=os.getenv("NEWSAPI_KEY"))
bitcoin_headlines = client.get_everything(q="bitcoin", language='en', page_size=100, sort_by='relevancy')

In [36]:
pprint(bitcoin_headlines)

{'articles': [{'author': 'Joel Khalili',
               'content': 'On March 12, Russian-Swedish national Roman '
                          'Sterlingov was found guilty of money laundering '
                          'conspiracy and other violations by a federal jury '
                          'in Washington, DC, for having operated Bitcoin Fog, '
                          'a ser… [+3654 chars]',
               'description': 'A jury convicted Roman Sterlingov of money '
                              'laundering this month. His defense team says it '
                              'will appeal, saying the crypto-tracing '
                              'technique at the heart of the case is '
                              '“pseudoscience.”',
               'publishedAt': '2024-03-27T11:00:00Z',
               'source': {'id': 'wired', 'name': 'Wired'},
               'title': 'The Science of Crypto Forensics Survives a Court '
                        'Battle—for Now',
               'ur